# #1 Project by Team 4

In [1]:
import sys
!{sys.executable} -m pip install category_encoders


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from category_encoders.one_hot import OneHotEncoder as ce_OneHotEncoder


In [3]:
from category_encoders.hashing import HashingEncoder

In [4]:
df = pd.read_csv('sample_4.csv')

In [5]:
df.drop(columns=['vin'], inplace=True, errors='ignore')

In [6]:
df['sale_date'] = (
    df['saledate']
      .astype(str)
      .str.replace(r' GMT.*', '', regex=True)  # strip timezone suffix
      .pipe(pd.to_datetime, errors='coerce')   # parse to datetime
)
df.drop(columns=['saledate'], inplace=True)

df['sale_year']      = df['sale_date'].dt.year
df['sale_month']     = df['sale_date'].dt.month
df['sale_day']       = df['sale_date'].dt.day
df['sale_dayofweek'] = df['sale_date'].dt.dayofweek

KeyError: 'saledate'

In [ ]:
num_cols = df.select_dtypes(include='number').columns.drop('sellingprice')
df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])

cat_cols = df.select_dtypes(include='object').columns.tolist()
df[cat_cols] = df[cat_cols].fillna('missing')


In [ ]:
cardinalities = df[cat_cols].nunique()
low_card_cols  = cardinalities[cardinalities <= 20].index.tolist()
high_card_cols = cardinalities[cardinalities  > 20].index.tolist()

print("Low-cardinality cols  (<=20 uniques):",  low_card_cols)
print("High-cardinality cols (>20 uniques):", high_card_cols)

In [ ]:
ohe = ce_OneHotEncoder(cols=low_card_cols, use_cat_names=True)
df    = ohe.fit_transform(df)

In [ ]:
hash_enc = HashingEncoder(cols=high_card_cols, n_components=20)
df       = hash_enc.fit_transform(df)

In [ ]:
X = df.drop(columns=['sellingprice'])
y = df['sellingprice']

print(X.head())

In [ ]:
df.to_csv('processed_sample_4.csv', index=False)